In [83]:
import pandas as pd
list_csv = ['data/kor/attraction_places.csv',
            'data/kor/hotel_places.csv',
            'data/kor/restaurant_places.csv',
            'data/eng/eng_attraction_places.csv',
            'data/eng/eng_hotel_places.csv',
            'data/eng/eng_restaurant_places.csv']
save_csv = ['data/attraction_places.csv',
            'data/hotel_places.csv',
            'data/restaurant_places.csv']

In [5]:
for csv in list_csv:
    df = pd.read_csv(csv,
                     names=['placeId', 'name_kor', 'location', 'class'],
                     encoding='euc-kr')
    print(len(df))

561
804
23150
564
899
12599


In [88]:
for i in range(3):
    df_kor = pd.read_csv(list_csv[i],
                     names=['placeId', 'name_kor', 'location', 'class'],
                     encoding='euc-kr')
    tmp_class = df_kor['class'][0]
    df_eng = pd.read_csv(list_csv[i+3],
                     names=['placeId', 'name_eng', 'location', 'class'],
                     encoding='euc-kr')
    df_eng = df_eng.iloc[:, [j for j, c in enumerate(df_eng.columns) if j != 4]]
    df_eng = df_eng.drop(columns=['location', 'class'])
    df_kor = pd.merge(df_kor, df_eng, how='outer')
    cols = df_kor.columns.tolist()
    cols = cols[3:4] + cols[0:2] + cols[-1:] + cols[2:3]
    df_kor = df_kor[cols]
    df_kor['class'] = df_kor['class'].fillna(tmp_class)
    df_kor.to_csv(save_csv[i], index=False)